<a href="https://colab.research.google.com/github/Johyeyoung/capstone-2021-35/blob/master/Transaction/KoBERT/KoBERT_fine_tunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 라이브러리 설치

In [ ]:
# 이는 구글 코랩으로 돌린 버전입니다. 그리고 기본 코드는 SKT Brain의 KoBERT를 그대로 가져왔고, 학습 및 테스트 데이터셋만 따로 준비한 것입니다.
# SKT Brain github 주소는 다음과 같습니다. https://github.com/SKTBrain/KoBERT

!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3 # 최신 버전으로 설치하면 "Input: must be Tensor, not str" 라는 에러 발생
!pip install torch

!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

## KoBERT 불러오기

In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

##GPU 사용 시
device = torch.device("cuda:0")
bertmodel, vocab = get_pytorch_kobert_model()

# 기본 Bert tokenizer 사용
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)


class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))
        

     |████████████████████████████████| 46.9MB 68kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 348kB 16.9MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595694 sha256=33e73e6b8f61599010522b90c4d5e9c33b9fc8b2cc88b5f4ec3d9b28fa09112a
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
     |████████████████████████████████| 1.2MB 14.2MB/s 
     |████████████████████████████████| 757kB 18.2MB/s 
     |████████████████████████████████| 3.0MB 47.7MB/s 
     |████████████████████████████████| 870kB 60.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=6e70e0e63694352c649adf0d1060d9cf8f22517a891e9534a079230a462a8b64
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f44

## 데이터 로드

In [16]:

# 학습용 데이터셋 불러오기
import pandas as pd
dataset_train1 = pd.read_csv('/content/drive/MyDrive/Trigger_list_내비_타이어.csv', encoding='cp949')
dataset_train1.head()


# 데이터 전처리
dataset_train1.drop(['type_num'], axis=1, inplace=True)
dataset_train1.head()
# positive 비율을 확인했다를 확인 1:1로 맞춰줘야 비율
# 같은 내용의 카테고리 합치기
dataset_train1["type"] = dataset_train1["type"].str.replace(pat=r'하이패스_카드분실', repl=r'하이패스_분실', regex=True)
dataset_train1["type"] = dataset_train1["type"].str.replace(pat=r'트렁크_파손', repl=r'트렁크_고장', regex=True)
dataset_train1["type"] = dataset_train1["type"].str.replace(pat=r'거치대_(분실|파손)', repl=r'내비게이션_파손', regex=True)
dataset_train1["type"] = dataset_train1["type"].str.replace(pat=r'내비게이션_인터넷연결불가', repl=r'내비게이션_고장', regex=True)
dataset_train1["type"] = dataset_train1["type"].str.replace(pat=r'타이어_파스/펑크\(후우\)', repl=r'타이어_파스/펑크', regex=True)
dataset_train1["type"] = dataset_train1["type"].str.replace(pat=r'타이어_파스/펑크\(후좌\)', repl=r'타이어_파스/펑크', regex=True)
dataset_train1["type"] = dataset_train1["type"].str.replace(pat=r'타이어_파스/펑크\(전우\)', repl=r'타이어_파스/펑크', regex=True)
dataset_train1["type"] = dataset_train1["type"].str.replace(pat=r'타이어_파스/펑크\(전좌\)', repl=r'타이어_파스/펑크', regex=True)
dataset_train1["type"] = dataset_train1["type"].str.replace(pat=r'블랙박스_탈거', repl=r'블랙박스_고장', regex=True)
dataset_train1["type"] = dataset_train1["type"].str.replace(pat=r'타이어_파스/펑크\(전우\)', repl=r'타이어_파스/펑크', regex=True)


dataset_train1['type'].unique()

# 대표 장애요인 추출
data1 = dataset_train1[dataset_train1['type'] == '타이어_파스/펑크']
data2 = dataset_train1[dataset_train1['type'] == '내비게이션_고장']
data3 = dataset_train1[dataset_train1['type'] == '블랙박스_고장']
data4 = dataset_train1[dataset_train1['type'] == '하이패스_분실']
data5 = dataset_train1[dataset_train1['type'] == '주행_차량떨림']

new_data = data1.append([data2, data3, data4, data5], sort=False)
new_data = pd.DataFrame(new_data)
new_data.head()

# 장애요인 라벨링
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(new_data['type'])
new_data['type'] = encoder.transform(new_data['type'])
new_data.head()


# 라벨링된 type 매핑 ex) {0: '타이어_파스/펑크', 1: '내비게이션_고장', 2: '블랙박스고장', 3: '하이패스 분실', 4: '주행_차량떨림'}
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
print(mapping)

# Train / Test set 분리
from sklearn.model_selection import train_test_split
train, test = train_test_split(new_data, test_size=0.2, random_state=42)
print("train shape is:", len(train))
print("test shape is:", len(test))


{0: '내비게이션_고장', 1: '블랙박스_고장', 2: '주행_차량떨림', 3: '타이어_파스/펑크', 4: '하이패스_분실'}
train shape is: 1341
test shape is: 336


In [26]:
type_num = train['type'].values.tolist()
question = train['문의 내용'].values.tolist()
train_list = [[question[i], type_num[i]] for i in range(len(type_num))]
# Train 리스트 비율 확인
print('Train ratio :', [type_num.count(i) for i in range(len(mapping))])

type_num = test['type'].values.tolist()
question = test['문의 내용'].values.tolist()
test_list = [[question[i], type_num[i]] for i in range(len(type_num))]
# Test 리스트 비율 확인
print('Test ratio: ', [type_num.count(i) for i in range(len(mapping))])


Train ratio : [862, 108, 80, 273, 18]
Test ratio:  [228, 22, 17, 66, 3]


In [23]:
for i in range(len(test_list)):
  print(train_list[i])

['타이어 파손 육안 확인 공기압 경고등 점등', 3]
['내비 작동 불가 인입', 0]
['내비게이션 인터넷 연결 안 된다고 인입', 0]
['운행 중 차가 잘 안 나가 확인 시 타이어 펑크나 있어 문의', 3]
['이전 문제 차량 환불 안돼서 문의', 3]
['내비가 안 켜져 인입', 0]
['내비게이션 화면이 꺼졌다 켜졌다 한다고 함', 0]
['tcs 재시작 해달라며 인입', 0]
['후방 블랙박스 떨어져 있음', 1]
['후 좌 타이어 공기압 낮음', 3]
['단순 고장', 3]
['공기압 경고 등 점 등 미확인', 3]
['내비게이션이 작동되지 않아 문의', 0]
['고객이 언급한 대로 상세 장애 내용 작성', 3]
['네트워크 연결 불가 장애 확인', 0]
['차량 시동 ON 상태로 엑셀 밟기 전 차량이 덜덜 떨림 엑셀 밟으면 괜찮아짐 차량 점검 요청', 2]
['내비 인터넷 연결 불가 인입', 0]
['내비게이션이 아예 켜지지 않는다며 인입', 0]
['블랙박스 녹화 안 된다고 나온다고 함', 1]
['내비 작동 불가 인입', 0]
['네비 네트워크 연결 오류', 0]
['네비 안돼서 문의', 0]
['이용차량 내비게이션 소리 나지 않음 볼륨 최대 같으나 소리 안남', 0]
['비접촉 주장 핸들이기 움', 3]
['차량을 운행 중인데 블랙박스가 떨어져 있어서 문의', 1]
['타이어 펑크 후 좌', 3]
['내비게이션 및 블루투스', 0]
['내비 위치가 아예 다른 곳으로 나온다고 함 사진 요청 불가', 0]
['내비게이션 작동 불가 문의', 0]
['차량 변경 후인 입', 0]
['TPMS 경고등 타이어 파손 육안 확인', 0]
['내비게이션 선이 다 빠져있고 내비 인터넷이 안된다 잠시 후 TMAP이 실행된다고 나오고 네트워크 문제 라고 한다', 0]
['내비게이션 인터넷 연결 불가', 0]
['내비게이션 재시작 받았지만 여전히 인터넷 연결 불가하여 연락드림', 0]
['내비게이션 인터넷 연결 불가로 문의', 0]
['워셔액 없음 사비로 결제하

In [8]:
# CUDA memory 부족 해결
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [9]:
# Setting parameters
max_len = 164 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5



data_train = BERTDataset(train_list, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(test_list, 0, 1, tok, max_len, True, False)

# pytorch용 DataLoader 사용
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 5, # softmax 사용 <- binary일 경우는 2
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)


model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

# 옵티마이저 선언
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # softmax용 Loss Function 정하기 <- binary classification도 해당 loss function 사용 가능

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
  
# 모델 학습 시작
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    


using cached model
using cached model
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
train shape is: 1341
test shape is: 336
using cached model
862
108
80
273
18
0
228
22
17
66
3
0


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:201: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


epoch 1 batch id 1 loss 1.6351953744888306 train acc 0.1875

epoch 1 train acc 0.5170521077283373


epoch 2 batch id 1 loss 1.0244672298431396 train acc 0.671875

epoch 2 train acc 0.7444257416081187


epoch 3 batch id 1 loss 0.6702820658683777 train acc 0.734375

epoch 3 train acc 0.8435304449648713


epoch 4 batch id 1 loss 0.5701027512550354 train acc 0.765625

epoch 4 train acc 0.8821843286494926


epoch 5 batch id 1 loss 0.5411780476570129 train acc 0.8125

epoch 5 train acc 0.8956869633099142


In [18]:
# 테스트 문장 예측
test_sentence = '네비게이션 분실로 인입.'
test_label = 0 # 장애 번호
# 라벨링된 type 매핑 ex) {0: '타이어_파스/펑크', 1: '내비게이션_고장', 2: '블랙박스고장', 3: '하이패스 분실', 4: '주행_차량떨림'}
print(mapping)
unseen_test = pd.DataFrame([[test_sentence, test_label]], columns = [['질문 내용', '장애요인']])
unseen_values = unseen_test.values
test_set = BERTDataset(unseen_values, 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=5)

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  out = model(token_ids, valid_length, segment_ids)
  print(out)
    


{0: '내비게이션_고장', 1: '블랙박스_고장', 2: '주행_차량떨림', 3: '타이어_파스/펑크', 4: '하이패스_분실'}


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


tensor([[ 3.3957, -0.8822, -1.1875, -0.2964, -1.8421]], device='cuda:0',
       grad_fn=<AddmmBackward>)

